In [1]:
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from dotenv import load_dotenv
_ = load_dotenv()

from langchain_core.messages import SystemMessage, HumanMessage
from agents.supervisor_agent import ChatSupervisorAgent
from agents.action_execution_agent import ActionExecutionAgent

Load Azure Models

In [2]:
from agents.llm_model import AzureModelProvider
provider = AzureModelProvider()

llm = provider.get_primary_model()
fast_llm = provider.get_light_model()

In [3]:
llm.invoke([SystemMessage(content="Ready to assist?")]).content

'Hello! How can I assist you today?'

Import Prompts

In [4]:
# Load the intent classifier prompt from the text file
with open(os.path.join(parent_dir, "prompts", "intent_classifier_prompt.txt"), "r") as f:
    intent_classifier_prompt = f.read()

# Load the action execution prompt from the text file
with open(os.path.join(parent_dir, "prompts", "action_execution_prompt.txt"), "r") as f:
    action_execution_prompt = f.read()


## Agent Testing

Action Execution

In [ ]:
# Removed inline class definition to use the imported ActionExecutionAgent from the file.
# This ensures we are testing the actual codebase.

In [5]:
action_agent = ActionExecutionAgent(llm, action_execution_prompt)

In [6]:
message = "What's my last payslip?"


test_state = {
    "messages": [HumanMessage(content=message)],
    "candidate_id": os.environ.get("KENTRO_TEST_CANDIDATE_ID"),
    "employee_number": os.environ.get("PLANBITION_TEST_EMPLOYEE_NUM"),
    "next_action": "ActionExecutionAgent",
    "retrieved_data": "",
    "error_message": "",
    "attempts": 0,
    "max_retries": 3
}

In [7]:
action_agent.run(test_state)

---ACTION EXECUTION AGENT: Starting run()---
---Invoking LLM (with tools)---
---Executing Tool: get_payslip---
---Executing Tool: get_payslip---
---Tool execution completed. Result length: 120---
---Tool execution completed. Result length: 120---


{'retrieved_data': "id=1223595 business_unit_id=10023 serial_number='25-46-001' frequency='W' entry_date='2025-11-19T00:12:52' amount=448.97",
 'error_message': None}

ChatSupervisor

In [ ]:
agent = ChatSupervisorAgent(model=llm, system_prompt=intent_classifier_prompt)

In [ ]:
# message = "what's my schedule for next week?"
message = "How can I call in sick?"
message = "What's my last payslip?"
# message = "What's Trump's first name?"

In [ ]:
test_state = {
    "messages": [HumanMessage(content=message)],
    "candidate_id": os.environ.get("KENTRO_TEST_CANDIDATE_ID"),
    "employee_number": os.environ.get("PLANBITION_TEST_EMPLOYEE_NUM"),
    "next_action": "",
    "retrieved_data": "",
    "error_message": "",
    "attempts": 0,
    "max_retries": 3
}

test_state

In [ ]:
# Run
result = agent.graph.invoke(test_state)

In [ ]:
result["retrieved_data"]

In [ ]:
agent.__getstate__